Remember to

```sh
sudo apt install ffmpeg libavcodec-extra
```

In [1]:
import sys
import os
import torch
import numpy as np
from tqdm import tqdm

# Get the absolute path of the project root
project_root = os.path.abspath("..")  # Adjust path if your notebook is deeper in directories

# Add project root to sys.path
sys.path.append(project_root)

# Now you can import dinov2
import dinov2
from dinov2.eval.setup import build_model_for_eval
from dinov2.configs import load_and_merge_config
from dinov2.utils.visualize import *

device = "cuda"
device = torch.device(device if torch.cuda.is_available() else "cpu")
print(device)

/home/lyk/Projects/dinov2/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/lyk/Projects/dinov2/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/home/lyk/Projects/dinov2/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


cuda


In [2]:
def main(video_path, model, output_path, threshold=0.6, device='cuda'):
    raw_tensor, input_tensor, fps = load_preprocess_video(video_path, target_size=448*1, patch_size=model.patch_size)  # 448 is multiple of patch_size (14)
    B, C, H, W, patch_size, embedding_dim, patch_num = print_video_model_stats(input_tensor, model)
    
    patch_embed_list = []
    with torch.no_grad():
        for i in tqdm(range(B), desc="Processing Frames", unit="frame"):
            frame = input_tensor[i].unsqueeze(0)
            patch_embed = get_patch_embeddings(model, frame)
            patch_embed_list.append(patch_embed)
            
    patch_embeds = np.vstack(patch_embed_list)  # (B, num_patches, embedding_dim)

    # Show progress for PCA processing
    print("Performing Two-Stage PCA...")
    reduced_embeds, reduced_fg_embeds, nums_of_fg_patches, masks = two_stage_pca(patch_embeds, threshold=threshold)

    # Saving video with progress
    print("Saving output video...")
    save_triple_video(raw_tensor, reduced_embeds, reduced_fg_embeds, nums_of_fg_patches, masks, patch_num, patch_size, output_path=output_path, fps=fps)

    print("Processing completed! ✅")
    


For crane, use b and 0.7.
For dog, use b and 0.7.
No registers.

In [ ]:
threshold=0.7
model_size = "b"

video_path = "./data/videos/dinov2_dog.mp4"
output_path = f"./data/outputs/embed_dog_{model_size}_{threshold}.mp4"


# Use `dinov2_vitb14_pretrain`
conf = load_and_merge_config(f'eval/vit{model_size}14_pretrain')
model = build_model_for_eval(conf, f'../dinov2/checkpoints/dinov2_vit{model_size}14_pretrain.pth')

main(video_path, model, output_path, threshold, device)

Video FPS: 1.00, Total Frames: 4, Duration: 4.00 seconds
Input tensor shape: Batch=4, Channels=3, Height=448, Width=448
Patch size: 14
Embedding dimension: 768
Number of patches of each image: 1024


Processing Frames: 100%|██████████| 4/4 [00:00<00:00, 37.10frame/s]

Performing Two-Stage PCA...


Num of foreground patches of image 0: 283
Num of foreground patches of image 1: 293
Num of foreground patches of image 2: 337
Num of foreground patches of image 3: 670
Total num of foreground patches: 1583
Explained variance ratio by PCA components: [0.11140188 0.09799817 0.06075843]
Saving output video...
Video saved to ./data/outputs/embed_crane_b_0.6.mp4
Processing completed! ✅
